In [1]:
import os
import shutil
from functools import partial
from glob import glob

import cv2
import helper
import numpy as np
from tqdm import tqdm

np.random.seed(2022)

In [2]:
perturbation = {}
perturbation["gaussian_pixel_noise"] = [
    (2 * i, partial(helper.gaussian_pixel_noise, std=2 * i)) for i in range(10)
]
perturbation["gaussian_blur"] = [
    (i, partial(helper.gaussian_blur, num_convolve=i)) for i in range(10)
]
perturbation["contrast_increase"] = [
    (i, partial(helper.scale_contrast, scale=i))
    for i in [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.1, 1.15, 1.20, 1.25]
]
perturbation["contrast_decrease"] = [
    (i, partial(helper.scale_contrast, scale=i))
    for i in [1.0, 0.95, 0.90, 0.85, 0.80, 0.60, 0.40, 0.30, 0.20, 0.10]
]
perturbation["brightness_increase"] = [
    (i, partial(helper.change_brightness, value=i)) for i in range(0, 50, 5)
]
perturbation["brightness_decrease"] = [
    (i, partial(helper.change_brightness, value=-i)) for i in range(0, 50, 5)
]
perturbation["occlusion"] = [
    (i, partial(helper.occlusion, edge_length=i)) for i in range(0, 50, 5)
]
perturbation["salt_and_pepper"] = [
    (i / 100, partial(helper.salt_and_pepper, rate=i / 100)) for i in range(0, 20, 2)
]

In [3]:
with tqdm(total=len(perturbation)) as pbar:
    for perturbation_type, perturbation_fn in perturbation.items():
        shutil.rmtree(os.path.join("dataset", perturbation_type), ignore_errors=True)
        for value, fn in perturbation_fn:
            for label_directory in glob(os.path.join("dataset", "test", "*")):
                label = label_directory.split("/")[-1]
                folder_dir = os.path.join(
                    "dataset", perturbation_type, str(value), label
                )
                os.makedirs(folder_dir)
                for image_directory in glob(os.path.join(label_directory, "*")):
                    image = helper.load_image(image_directory)
                    image = fn(image)
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    image_name = image_directory.split("/")[-1]
                    image_save_dir = os.path.join(folder_dir, image_name)
                    if not cv2.imwrite(image_save_dir, image):
                        raise ValueError
        pbar.update()

100%|██████████| 8/8 [02:23<00:00, 17.92s/it]
